# fct_operator_hourly_summary

In [1]:
import datetime
import sys

import _operator_grain_route_dir_visuals as _report_operator_visuals
import _operator_grain_scheduled_service
import _sql_query
import google.auth
import pandas as pd
from google.cloud import bigquery
from IPython.display import HTML, Image, Markdown, display, display_html
from loguru import logger
from omegaconf import OmegaConf
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SEGMENT_GCS

import altair as alt
import geopandas as gpd
from shapely import wkt

from omegaconf import OmegaConf
readable_dict = OmegaConf.load("new_readable.yml")

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_name = "City and County of San Francisco"

In [4]:
url = "gs://calitp-analytics-data/data-analyses/gtfs_digest/processed/fct_operator_hourly_summary_2025_12.parquet"

In [5]:
df = pd.read_parquet(url)

In [6]:
df.sample(3)

,Analysis Name,Date,Day Type,Departure Hour,N Trips
20471,City of Needles,02-2025,Weekday,9,1
41259,Mission Bay Transportation Management Agency,01-2025,Weekday,16,22
7818,City of Banning,01-2025,Weekday,12,6


In [7]:
df["Departure Hour"].unique()

<IntegerArray>
[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
 19, 20, 21, 22, 23]
Length: 24, dtype: Int64

In [8]:
df["Date"].unique()

array(['01-2025', '02-2025', '03-2025', '04-2025', '05-2025', '06-2025',
       '07-2025', '08-2025', '09-2025', '10-2025', '11-2025'],
      dtype=object)

In [9]:
sf_only = df.loc[df["Analysis Name"] == analysis_name]

In [10]:
def create_hourly_summary(df: pd.DataFrame, day_type: str):
    
    chart_dict = readable_dict.hourly_summary
    df2 = df.loc[df["Day Type"] == "Saturday"]
    df2["Date"] = df["Date"].astype(str)
    
    date_list = list(df2["Date"].unique())
    
    date_dropdown = alt.binding_select(
        options=date_list,
        name="Dates: ",
    )
    xcol_param = alt.selection_point(
        fields=["Date"], value=date_list[0], bind=date_dropdown
    )

    chart = (
        (
            alt.Chart(df2)
            .mark_line(size=3)
            .encode(
                x=alt.X(
                    "Departure Hour",
                    title="Departure Hour",
                    axis=alt.Axis(
                        labelAngle=-45,
                    ),
                ),
                y=alt.Y(
                    "N Trips",
                    title="N Trips",
                ),
            )
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
    
    bg = _operator_grain_scheduled_service.create_bg_service_chart()
    
    chart = (chart + bg).properties(
    resolve=alt.Resolve(
        scale=alt.LegendResolveMap(color=alt.ResolveMode("independent"))
    )
)
    chart = _report_operator_visuals.configure_chart(
    chart,
    width=400,
    height=250,
    title=f"{chart_dict.title} {day_type}",
    subtitle=chart_dict.subtitle)
    
    return chart

In [11]:
create_hourly_summary(sf_only, "Saturday")

/tmp/ipykernel_3299/2827727319.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Date"] = df["Date"].astype(str)


alt.LayerChart(...)

In [12]:
create_hourly_summary(sf_only, "Sunday")

/tmp/ipykernel_3299/2827727319.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Date"] = df["Date"].astype(str)


alt.LayerChart(...)

In [13]:
create_hourly_summary(sf_only, "Weekday")

/tmp/ipykernel_3299/2827727319.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Date"] = df["Date"].astype(str)


alt.LayerChart(...)